$$\mbox{Cours optimisation (IG2) : travaux pratiques}$$
$$\mbox{Programmation Linéaire, Programmation en Nombres Entiers, Programmation Semi-définie Positive, Jeux}$$

We are going to use Julia. There are different options to do that.
- You can install Julia on your computer (you might need an editor such as atom) and execute evrything locally. You can just copy some parts of the code given in the html file.
- In addition to that, you can also install jupyter on your machine and then open the file *.ipynb
- Use google colab, in this case, you need to have a google account (and you d'ont need to install Julia). The file is stored on the Google drive, and programs are executed on a virtual machine.  You have to open the *.ipynb and run the installation cell below. 

In [1]:
# Installation cell
%%capture
%%shell
if ! command -v julia 3>&1 > /dev/null
then
    wget -q 'https://julialang-s3.julialang.org/bin/linux/x64/1.7/julia-1.7.2-linux-x86_64.tar.gz' \
        -O /tmp/julia.tar.gz
    tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
    rm /tmp/julia.tar.gz
fi
julia -e 'using Pkg; pkg"add IJulia; precompile;"'
echo 'Done'

After you run the first cell (the cell directly above this text), go to Colab's menu bar and select **Edit** and select **Notebook settings** from the drop down. Select *Julia 1.7* in Runtime type. You can also select your prefered harwdware acceleration (defaults to GPU). 

<br/>You should see something like this:

> ![Colab Img](https://raw.githubusercontent.com/Dsantra92/Julia-on-Colab/master/misc/julia_menu.png)

<br/>Click on SAVE
<br/>**We are ready to get going**





In [1]:
VERSION


v"1.8.5"

# Attaques et mesures de sécurité 

Un opérateur ou un administrateur d'un réseau informatique dispose d'une panoplie de mesures de sécurité qu'il pourrait mettre en place pour se protéger contre les attaques informatiques. Imaginons que $N_m$ mesures numérotées $1, ..., N_m$ sont disponibles. Comme la mise en place de trop de mesures induit des surcoûts (financiers, détérioration du service, etc.), l'opérateur devrait choisir $k_m$ mesures parmi ces $N_m$ en essayant de minimiser l'impact des attaques qu'il pourrait ne pas détecter. Une entité malveillante cherche justement à attaquer le réseau et dispose d'une panoplie d'attaques. Disons que $N_a$ attaques sont possibles et que l'attaquant compte en sélectionner $k_a$ attaques en tentant de maximiser l'impact de ses attaques. Utilisons le vecteur $x$ de taille $N_m$ pour identifier les mesures mises en place ($x_i = 1$ si la mesure $i$ est mise en place et $0$ sinon). Si une attaque du type $j$ est lancée, on peut  estimer que la probabilité pour que la mesure de sécurité $i$ permette de l'intercepter est égale à $p_{i,j}$. En faisant une hypothèse d'indépendence, l'attaque $j$ n'est pas interceptée avec la probabilité $p_j = \prod_{i = 1}^{N_m} (1 - x_i p_{i j})$. On estime que l'impact de l'attaque $j$ s'exprime sous la forme de $w_j \log{p_j}$. Ainsi, d'une part il croît avec la probabilité de non-interception $p_j$. D'autre part, il dépend de $j$ à travers le terme $w_j$. Enfin, il s'exprime sous la forme de $\sum_{i = 1}^{N_m} x_i w_j \log{(1 - p_{i j})}$ qu'on pourra écrire comme $\sum_{i = 1}^{N_m} x_i w_{i j}$. Si on utilise le vecteur $y$ de taille $N_a$ pour identifier les attaques, l'objectif de l'attaquant est donc de maximiser $\sum_{i =1}^{N_m} \sum_{j = 1}^{N_a} x_i y_j w_{i j}$ alors que le défenseur cherche à minimiser cette quantité. Ainsi nous obtenons ici ce qu'on appelle  $\underline{\mbox{un jeu à somme nulle}}$. 

Supposons que l'adminstrateur choisit ses $k_m$ mesures, et ensuite l'attaquant (qui a la possibilité de prendre connaissance des mesures déployées) choisit ses $k_a$ attaques (on a ici ce qu'on appelle un jeu du type Stackelberg avec un "leader" et un "follower"). Ainsi, l'objectif de l'opérateur est de résoudre le problème suivant : 
\begin{equation}
\min_{x \in \{0,1\}^{N_m} \\ \sum_{i} x_i = k_m} \max_{y \in \{0,1\}^{N_a} \\ \sum_{j} y_j = k_a} \sum_{i = 1}^{N_m}  \sum_{j = 1}^{N_a}   x_i y_j w_{i j}
\end{equation}

## Question 1
En remarquant que le problème intérieur (le max) peut être remplacé par un programme linéaire (c'est à dire que l'on peut relacher les contraintes d'intégrité), démontrez qu'il est possible d'exprimer le problème de l'opérateur comme suit :

\begin{equation}
\begin{array}{ll}
& \min k_a \gamma + \sum_{j = 1}^{N_a} \pi_j \\
& x_i \in \{0,1\}, \forall i \in \{1, ..., N_m \} \\
& \sum_{i = 1}^{N_m} x_i = k_m \\
& \pi_j \geq 0, \forall j \in \{1, ..., N_a \} \\
& \pi_j + \gamma \geq \sum_{i = 1}^{N_m} x_i w_{i j}, \forall j \in \{1, ..., N_a \} \\
& \gamma \in R 
\end{array}
\end{equation}

La fonction suivante permet d'implémenter le MIP précédent.  Le résultat obtenu pour une matrice particulière W est donné.  

In [3]:
import Pkg
Pkg.add("JuMP")
Pkg.add("GLPK")
using JuMP
using GLPK

function min_max_int(w, n1 ,k1 , n2, k2)

model=Model(GLPK.Optimizer)

@variable(model, x[1:n1], Bin)

@variable(model, gamma)

@variable(model, pi[1:n2]>=0)

@constraint(model,somme, sum(x[i] for i in 1:n1) ==k1)

@constraint(model,contrainte[j in 1:n2],pi[j] - sum(w[i,j]*x[i] for i in 1:n1)+ gamma>=0)

@objective(model, Min, sum(pi[j] for j in 1:n2) + k2*gamma)

print(model)

optimize!(model)
    
println("Solution optimale : ")
    
for i in 1:n1
    println(" x[", i, "] = ",  value(x[i]))
 end
        
return objective_value(model)
end

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
   Installed IrrationalConstants ── v0.1.1
   Installed DiffRules ──────────── v1.12.2
   Installed Bzip2_jll ──────────── v1.0.8+0
   Installed CodecBzip2 ─────────── v0.7.2
   Installed DiffResults ────────── v1.1.0
   Installed BenchmarkTools ─────── v1.3.2
   Installed MutableArithmetics ─── v1.1.0
   Installed SpecialFunctions ───── v2.1.7
ERROR: Data Error : jl_9jVPfIlEimF4~
   Installed StaticArraysCore ───── v1.4.0
   Installed NaNMath ────────────── v1.0.1
   Installed OrderedCollections ─── v1.4.1
   Installed TranscodingStreams ─── v0.9.11
   Installed StaticArrays ───────── v1.5.12
   Installed ForwardDiff ────────── v0.10.34
   Installed ChainRulesCore ─────── v1.15.6
┌ Warning: failed to extract archive downloaded from https://pkg.julialang.org/package/4076af6c-e467-56ae-b986-b466b2749572/821341783388cd26bbb9bab4ad9d0992e78259a4
└ @ Pkg.Operations /buildworker/worker/package_linux

min_max_int (generic function with 1 method)

Pour tester, prenons les valeurs suivantes :

In [4]:
global Nm = 6
global Na = 8
global km = 3
global ka = 3
global W = zeros(Float64,Nm,Na)

W =[ -4.0   0.0  -9.0   -1.0   -2.0  -9.0  -4.0   -5.0   
 -10.0  -1.0  -7.0    0.0    0.0  -5.0  -6.0   -5.0
  -3.0  -9.0  -9.0   -5.0    0.0  -7.0  -3.0  -10.0
 -10.0   0.0  -4.0    0.0  -10.0  -7.0  -6.0   -2.0
  -9.0  -7.0  -8.0    0.0   -1.0  -6.0  -5.0    0.0
  -3.0  -3.0  -5.0  -10.0   -9.0  -1.0  -4.0   -8.0]

6×8 Matrix{Float64}:
  -4.0   0.0  -9.0   -1.0   -2.0  -9.0  -4.0   -5.0
 -10.0  -1.0  -7.0    0.0    0.0  -5.0  -6.0   -5.0
  -3.0  -9.0  -9.0   -5.0    0.0  -7.0  -3.0  -10.0
 -10.0   0.0  -4.0    0.0  -10.0  -7.0  -6.0   -2.0
  -9.0  -7.0  -8.0    0.0   -1.0  -6.0  -5.0    0.0
  -3.0  -3.0  -5.0  -10.0   -9.0  -1.0  -4.0   -8.0

In [5]:
min_max_int(W, Nm ,km , Na , ka)

Solution optimale : 
 x[1] = 0.0
 x[2] = 0.0
 x[3] = 1.0
 x[4] = 1.0
 x[5] = 0.0
 x[6] = 1.0


-40.00000000000001

Supposons maintenant que l'attaquant joue en premier en décidant des $k_a$ attaques qu'il doit mener et l'opérateur pourra réagir en choisissant les meilleures mesures de sécurité. Il s'agit donc de résoudre le problème suivant : 
\begin{equation}
\max_{y \in \{0,1\}^{N_a} \\ \sum_{j} y_j = k_a}  \min_{x \in \{0,1\}^{N_m} \\ \sum_{i} x_i = k_m}  \sum_{i = 1}^{N_m}  \sum_{j = 1}^{N_a}   x_i y_j w_{i j}
\end{equation}
## Question 2
Montrez que le problème ci-dessus peut être résolu en utilisant la fonction précédemment implémentée (min_max_int). Plus précisément, l'objectif optimal est donné par (-1)*min_max_int(-W',Na,ka,Nm, km)

In [6]:
(-1)*min_max_int(-W',Na,ka,Nm, km)

Solution optimale : 
 x[1] = 0.0
 x[2] = 0.0
 x[3] = 0.0
 x[4] = 1.0
 x[5] = 0.0
 x[6] = 1.0
 x[7] = 1.0
 x[8] = 0.0


-44.0

## Question 3
On remarque que le min_max est supérieur au max_min. Montrez que ceci est vrai tout le temps (pour toute matrice $W$). Ainsi dans ce type de jeux, $\underline{\mbox{on n'a pas intérêt à jouer en premier}}$ ...

## Question  4
Supposons qu'une force maléfique suprème peut simulatanément fixer $x$ et $y$ de telle manière que $\sum_{i = 1}^{N_m}  \sum_{j = 1}^{N_a}   x_i y_j w_{i j}$ soit maximal. Il s'agit en quelque sorte de résoudre un problème du type max_max.  Montrez que l'on peut déterminer $x$ et $y$ grâce à la fonction ci-dessous :

In [7]:
function max_max_int(w, n1 ,k1 , n2, k2)

model=Model(GLPK.Optimizer)

@variable(model, y[1:n2],Bin)
@variable(model, x[1:n1], Bin)
@variable(model, z[1:n1,1:n2], Bin)

@constraint(model,somme_j[j in 1:n2], sum(z[i,j] for i in 1:n1) ==k1*y[j])
@constraint(model,somme_i[i in 1:n1], sum(z[i,j] for j in 1:n2) ==k2*x[i])
@constraint(model,somme_x, sum(x[i] for i in 1:n1) ==k1)
@constraint(model,somme_y, sum(y[j] for j in 1:n2) ==k2)

@objective(model, Max, sum(w[i,j]*z[i,j] for i in 1:n1,j in 1:n2))

print(model)

optimize!(model)

return objective_value(model)

end

max_max_int (generic function with 1 method)

In [8]:
max_max_int(W,Nm ,km , Na , ka)

-12.0

## Question 5

Supposons mainteant qu'une force bénefique suprème peut simulatanément fixer $x$ et $y$ de telle manière que $\sum_{i = 1}^{N_m}  \sum_{j = 1}^{N_a}   x_i y_j w_{i j}$ soit minimal. Il s'agit donc ici de calculer un min_min. 
Montrez que l'objectif optimal peut être obtenu comme suit :

In [9]:
(-1)*max_max_int(-W',Na,ka,Nm,km)

-67.0

## Question 6

Expliquez pourquoi on a $\max \max \geq \min \max \geq \max \min \geq \min \min$.

## Question 7

La stratégie qui consiste à fixer la valeur de chaque $x_i$ à $1$ ou $0$ est dite pure. L'opérateur (ainsi que l'attaquant) peuvent avoir des stratégies probabilistes mixtes où les $x_i$ et les $y_j$ peuvent prendre des valeurs fractionnaires. Ainsi, pour une valeur fractionnaire $x_i$, l'opérateur décide d'adopter la mesure $i$ avec une probabilité $x_i$ et de ne pas l'adpoter avec la probabilité $1 - x_i$. L'attaquant fait de même en ce qui concerne les attaques. ces strtatégies sont dites des stratégies mixtes puisqu'elle combinent des stratégies pures. Si l'opérateur joue en premier, il devra alors résoudre une version continue du problème (2). Ecrivez vous même un code pour la fonction  min_max_frac(w, n1 ,k1 , n2, k2). 

Appliquez votre fonction et vérifiez que vous obtenez un impact optimal de 
-42.18377976190476

Utilisez la fonction que vous venez de coder pour maximiser l'impact lorsque l'attaquant commence à jouer (vous pouvez par exemple appliquer (-1)*min_max_frac(-W',Na,ka,Nm, km)). 

Qu'est ce qu'on remarque ??  Tentez une explication en faisant le rapprochement avec le théroème de Sion, ou en utilisant la dualité....

En fait, on vient d'obtenir ce qu'on appelle un  $\underline{\mbox{équilibre de Nash}}$   (une stratégie mixte optimale  pour chacun des 2 jeux telle que aucun des 2 joueurs n'a intérêt à en dévier....)

## Question 8
Nous allons maintenant étudier des relaxations continues pour le problème max_max (qu'on a résolu à l'aide de la fonction max_max_int). Naturellment, des relaxations seront aussi déduites pour le problème min_min. Commençons par cette première relaxation :

In [11]:
function max_max_int_relax1(w, n1 ,k1 , n2, k2)

model=Model(GLPK.Optimizer)

@variable(model, 0<=y[1:n2]<=1)
@variable(model, 0<=x[1:n1]<=1)
@variable(model, 0<=z[1:n1,1:n2]<=1)

@constraint(model,somme_j[j in 1:n2], sum(z[i,j] for i in 1:n1) ==k1*y[j])
@constraint(model,somme_i[i in 1:n1], sum(z[i,j] for j in 1:n2) ==k2*x[i])
@constraint(model,somme_x, sum(x[i] for i in 1:n1) ==k1)
@constraint(model,somme_y, sum(y[j] for j in 1:n2) ==k2)

@objective(model, Max, sum(w[i,j]*z[i,j] for i in 1:n1,j in 1:n2))

print(model)

optimize!(model)

return objective_value(model)

end

max_max_int_relax1(W,Nm ,km , Na , ka)



-1.0000000000000013

Montrez la validité de cette relaxation. 

## Question 9
Allons un peu plus loin en rajoutant quelques inégalités valides. 

In [12]:
function max_max_int_relax2(w, n1 ,k1 , n2, k2)

model=Model(GLPK.Optimizer)

@variable(model, 0<=y[1:n2]<=1)
@variable(model, 0<=x[1:n1]<=1)
@variable(model, 0<=z[1:n1,1:n2]<=1)

@constraint(model,somme_j[j in 1:n2], sum(z[i,j] for i in 1:n1) ==k1*y[j])
@constraint(model,somme_i[i in 1:n1], sum(z[i,j] for j in 1:n2) ==k2*x[i])
@constraint(model,somme_x, sum(x[i] for i in 1:n1) ==k1)
@constraint(model,somme_y, sum(y[j] for j in 1:n2) ==k2)

@objective(model, Max, sum(w[i,j]*z[i,j] for i in 1:n1,j in 1:n2))

@constraint(model, RLT1[i in 1:n1,j in 1:n2],z[i,j]-x[i]<=0)
@constraint(model, RLT2[i in 1:n1,j in 1:n2],z[i,j]-y[j]<=0)

print(model)
optimize!(model)

return objective_value(model)

end

max_max_int_relax2(W,Nm ,km , Na , ka)

-8.000000000000023

Montrez la validité de cette relaxation et observez l'apport de ce renforcement par rapport à la relaxation précédente.

## Question 10
Nous allons renforcer la relaxation à l'aide de contraintes SDP. Nous considérons la matrice $U$ symétrique de taille $N_a + N_m + 1$  qui représente le produit $\left(\begin{array}{r} & x \\ & y \\ & 1  \end{array}\right) \times \left(\begin{array}{l}  & x \\ & y \\ & 1 \\  \end{array} \right)^T$. En utilisant le lemme du complément de Schur, montrez qu'écrire que  $U$ est SDP est équivalent à exiger que la matrice principale de taille $N_a + N_m$ (en haut à gauche) définie pat   $U[1:N_a+N_m,1:N_a+N_m]$ vérifie $U[1:N_a+N_m,1:N_a+N_m] -   \left(\begin{array}{l}  & x \\ & y  \\  \end{array} \right) \left(\begin{array}{l}  & x \\ & y  \\  \end{array} \right)^T$ est SDP. Ainsi la contrainte $U =  \left(\begin{array}{r} & x \\ & y \\ & 1  \end{array}\right) \times \left(\begin{array}{l}  & x \\ & y \\ & 1 \\  \end{array} \right)^T$ est relachée en la contrainte convexe $U[1:N_a+N_m,1:N_a+N_m] -   \left(\begin{array}{l}  & x \\ & y  \\  \end{array} \right) \left(\begin{array}{l}  & x \\ & y  \\  \end{array} \right)^T$ est SDP,  qui est donc équivalente à $U$ est SDP sachant que la dernière colonne de $U$ est égale au vecteur $\left(\begin{array}{r} & x \\ & y \\ & 1  \end{array}\right)$. 
Nous allons donc implémenter et résoudre une relaxation intégrant cette contrainte SDP. Nous avons besoin d'un solveur qui traite ce type de contrainte. Nous pouvons par exemple utiliser le solveur SCS.

In [13]:
Pkg.add("SCS")
using SCS

   Resolving package versions...
   Installed MKL_jll ───────── v2022.2.0+0
   Installed OpenBLAS32_jll ── v0.3.17+0
   Installed SCS_jll ───────── v3.2.1+0
   Installed SCS_GPU_jll ───── v3.2.1+0
   Installed Requires ──────── v1.3.0
   Installed SCS_MKL_jll ───── v3.2.2+0
   Installed IntelOpenMP_jll ─ v2018.0.3+2
   Installed SCS ───────────── v1.1.3
    Updating `~/.julia/environments/v1.7/Project.toml`
  [c946c3f1] + SCS v1.1.3
    Updating `~/.julia/environments/v1.7/Manifest.toml`
  [ae029012] + Requires v1.3.0
  [c946c3f1] + SCS v1.1.3
  [1d5cc7b8] + IntelOpenMP_jll v2018.0.3+2
  [856f044c] + MKL_jll v2022.2.0+0
  [656ef2d0] + OpenBLAS32_jll v0.3.17+0
  [af6e375f] + SCS_GPU_jll v3.2.1+0
  [3f2553a9] + SCS_MKL_jll v3.2.2+0
  [f4f2fc5b] + SCS_jll v3.2.1+0
  [4af54fe1] + LazyArtifacts
Precompiling project...
  ✓ Requires
  ✓ IntelOpenMP_jll
  ✓ OpenBLAS32_jll
  ✓ MKL_jll
  ✓ SCS_GPU_jll
  ✓ SCS_jll
  ✓ SCS_MKL_jll
  ✓ SCS
  8 dependencies successfully precompiled in 38 seconds (50

In [14]:
function max_max_int_relax3(w, n1 ,k1 , n2, k2)

model=Model(SCS.Optimizer)

@variable(model, 0<=y[1:n2]<=1)
@variable(model, 0<=x[1:n1]<=1)
@variable(model, U[1:n1+n2+1,1:n1+n2+1],PSD)

@constraint(model, bornes[i in 1:n1+n2+1,j in 1:n1+n2+1], 0<=U[i,j]<=1)

@constraint(model,type1[i in 1:n1],U[i,n1+n2+1]==x[i])
@constraint(model,type2[j in 1:n2],U[j+n1,n1+n2+1]==y[j])

@constraint(model,U[n1+n2+1,n1+n2+1]==1)

@constraint(model,somme_j[j in 1:n2], sum(U[i,j+n1] for i in 1:n1) ==k1*y[j])
@constraint(model,somme_i[i in 1:n1], sum(U[i,j+n1] for j in 1:n2) ==k2*x[i])

@constraint(model,identify_x[i in 1:n1], U[i,i]==x[i])
@constraint(model,identify_y[j in 1:n2], U[j+n1,j+n1]==y[j])


@constraint(model,somme_x, sum(x[i] for i in 1:n1) ==k1)
@constraint(model,somme_y, sum(y[j] for j in 1:n2) ==k2)

@constraint(model,RLT3[i in 1:n1],sum(U[i,j+n1] for j in 1:n2) ==k2*x[i])
@constraint(model,RLT4[j in 1:n2],sum(U[j+n1,i] for i in 1:n1) ==k1*y[j])

@constraint(model, RLT1[i in 1:n1,j in 1:n2],U[i,j+n1]-x[i]<=0)
@constraint(model, RLT2[i in 1:n1,j in 1:n2],U[i,j+n1]-y[j]<=0)

@constraint(model,RLT5[i in 1:n1],sum(U[i,l] for l in 1:n1) ==k1*x[i])
@constraint(model,RLT6[j in 1:n2],sum(U[j+n1,l+n1] for l in 1:n2) ==k2*y[j])

@constraint(model, RLT7[i in 1:n1,l in 1:n1],U[i,l]-x[i]<=0)
@constraint(model, RLT8[l in 1:n2,j in 1:n2],U[l+n1,j+n1]-y[j]<=0)


@objective(model, Max, sum(w[i,j]*U[i,j+n1] for i in 1:n1,j in 1:n2))

print(model)

optimize!(model)

return objective_value(model)

end
max_max_int_relax3(W,Nm ,km , Na , ka)

------------------------------------------------------------------
	       SCS v3.2.1 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
------------------------------------------------------------------
problem:  variables n: 134, constraints m: 867
cones: 	  z: primal zero / dual free vars: 73
	  l: linear vars: 674
	  s: psd vars: 120, ssize: 1
settings: eps_abs: 1.0e-04, eps_rel: 1.0e-04, eps_infeas: 1.0e-07
	  alpha: 1.50, scale: 1.00e-01, adaptive_scale: 1
	  max_iters: 100000, normalize: 1, rho_x: 1.00e-06
	  acceleration_lookback: 10, acceleration_interval: 10
lin-sys:  sparse-direct-amd-qdldl
	  nnz(A): 1395, nnz(P): 0
------------------------------------------------------------------
 iter | pri res | dua res |   gap   |   obj   |  scale  | time (s)
------------------------------------------------------------------
     0| 1.27e+01  6.35e+00  9.97e+02 -4.44e+02  1.00e-01  1.85e-03 
   250| 8.89e-03  2.35e-03  2.56e-02  1.12e+01  4.30e-01  3.07e-02 
  

-11.287239556348114

Implémentez la fonction max_max_int_relax3 et montrez la validité de la relaxation sous-jacente. On voit que l'optimum obtenu n'est pas loin de l'optimum entier (max_max_int). Essayons encore de nous en rapprocher.  

## Question 11

Quand on cherche à linéariser le produit de variables binaires (par exemple $U_{i,l} = x_i \times x_l$), on peut rajouter l'inégalité valide $U_{i, l} \geq x_i + x_l - 1$.  En rajoutant ce type de contraintes, nous obtenons une nouvelle relaxation qui est légèrement meilleure.

Modifiez donc la fonction précédente pour résoudre la nouvelle relaxation (définissez une nouvelle fonction function max_max_int_relax4(w, n1 ,k1 , n2, k2) qui intègre les nouvelles contraintes ainsi que celle déjà considérées dans function max_max_int_rela3...)

Résolvez... Quel est l'optimum de la relaxation ? 

 



## Question 12
   Faites  varier les tailles de l'instance et générer des instances aléatorires  (par exemple, à l'aide de la commande W = float(rand(-10:0,Nm,Na))). Observez le gap des 4 relaxations et les temps de calcul de max_min_int comparé à celui de chacune des 4 relaxations  (on pourra par exemple utiliser la commande   time = @elapsed max_max_int_relax4(W,Nm ,km , Na , ka)  pour mesurer le temps pris pour résoudre la quatrième relaxation).